## Lab 3
### UGBA 96: Data and Decisions, Fall 2018

<br>

This lab is designed to be completed in class. However, in case you need additional time, this assignment is due **Tuesday, October 16th at 11:59pm**.

The lab will be graded for **completion**. Lab office hours are held by Connector Assistants on Tuesdays after labs from 1-3pm in the DS Nexus in Moffitt.

## What's Advertising Content Worth?

<img src="credit_indemnity.png" alt="Drawing" style="width: 300px;"/>

Firms spend billions of dollars on advertising, but there is surprisingly little quantitative evidence on its effectiveness in persuading customers. What works? What doesn't work?

In this lab we will evaluate the advertising experiment introduced in lecture. The experiment was conducted by Credit Indemnity, one of the largest micro-lenders in South Africa. The lender offers short-term, high-interest, uncollateralized credit for people without other access to credit. The typical loan is about $150, about a third of the borrower's monthy income. Most of the loans last about four months.

Credit Indemnity performed an experiment in which they sent credit offers by mail to more than 50,000 previous customers. In this experiment Credit Indemnity randomly varied the offers in several ways, including the interest rates. But many of the variations were purely cosmetic, including whether the offer letter refers to the offered interest rate as 'special' or 'low' or whether it included a photo of a man or a woman.

In this lab, we will analyze the experiment data to determine which features of the offer letter made a difference in getting customers to apply for a loan.

---

### Table of Contents

1. [Describing the Experiment and Data](#experiment)<br>
2. [Measuring Treatment Effects](#treatment)<br>
3. [Are These Effects Statistically Significant?](#tests)<br>
4. [What Matters for Advertising Content?](#conclusion)<br>


**Dependencies:**

In [ ]:
from datascience import *
import numpy as np
import pandas as pd

#These lines set up graphing capabilities.
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from client.api.notebook import Notebook
ok = Notebook('lab3.ok')
_ = ok.auth(inline=True) 

## Section 1: Describing the Experiment Data <a id='experiment'></a>

Credit Indemnity randomized advertising content and the interest rate in actual offers to 53,000 former
clients.

Here's an example of a loan offer that was mailed as part of the experiment:

<img src="example_flyer.png" alt="Drawing" style="width: 400px;"/>

**Run the cell below to read in data from the experiment.**

In [ ]:
experiment = Table.read_table('loan_ads.csv')
experiment.show(5)

Below we describe how each column is defined. Each column represents a customer (indexed by `id`). 

* `risk`: The lender's assessment for how risky it is to lend to a particular customer. Customers that are less likely to pay back the loan are more risky. `risk` takes three values: 'Low', 'Medium', and 'High'.
* `offer_rate`: The interest rate on the loan offered to the customer.
* `photo_female`: An indicator whether the offer letter includes a photo of a woman.
* `photo_none`: An indicator for whether the offer letter includes no photo.
* `intshown`: An indicator for whether the offter letter explicitly shows the interest rate.
* `suggest_use`: An indicator for whether the offer letter suggests a specific use for the loan.
* `special_offer`: An indicator for whether the offer letter advertises the offered interest rate as 'special' or 'low'.

The outcome we will focus on is `applied`, an indicator for whether a customer applies for a loan. The mean of `applied` is 8.5%.

**Run the cell below to confirm the mean of `applied`**.

In [ ]:
#calculate mean of `applied`
experiment.column('applied').mean()

In the example offer letter shown above, the `offer_rate` is 3.99, `intshown` is zero, `suggest_use` is 1 ('You can use this cash to buy an appliance...'), and `special_offer` is one ('A special rate for you').

## Section 2: Measuring Treatment Effects <a id='treatment'></a>

In each subsection below, you will measure the causal effect for one of several treatments that Credit Indemnity tested in their experiment.

### Loan Applications and Interest Rates

In this section you will measure the causal effect of `offer_rate`, the interest rate offered in the letter, on customer application rates.

What is an **interest rate**? You can think of the interest rate as the the price of the loan--the higher the interest rate, the more a customer will generally have to pay. In this case, the interest rate is a percentage of the principal balance that the borrower must pay each month as a fee. For example, consider the four month loan advertised in the letter above, where the interest rate is 3.99. The borrowed amount is 1000 rand (the South African currency). In addition to that initial amount, the borrower pays about 40 ($\approx 0.0399 \times 1000$) rand each month in interest.

By contrast to the other letter features we will investigate, `offer_rate` is the only feature where there is a clear *economic* reason for customers to care about the feature.

The set of interest rates a customer may get depends on that customer's assessed risk level. Customers that the bank perceives have a **high** risk of not repaying their loan face higher interest rates than customers that the bank perceives are **low** risk. But among customers at a given risk level, `offer_rate` can take many values and is **randomly assigned**. Random assignment will allow us to measure the causal relationship between `offer_rate` and loan application rates.

**Run the cell below to show how `offer_rate` varies with `risk`.**

In [ ]:
experiment.group('risk', np.mean).select('risk', 'offer_rate mean')

Among customers at a given risk level, `offer_rate` is randomly assigned.

**Q1.** Plot a histogram of `offer_rate` among **'High'** risk customers.

In [ ]:
#write code here

Next, we will investigate the causal relationship between `offer_rate` and `applied`. By contrast to past 'treatments' we have studied, `offer_rate` takes more than two values. This means there are more than two potential outcomes for each customer; in fact, there is a distinct potential outcome for each value of `offer_rate`. In this case, a natural way to summarize the causal effect of `offer_rate` on `applied` is to plot the relationship between the two variables.

Before we do that, what do you expect to find?

**Q2.** Do you expect loan application rates to be related to `offer_rate`? If so, do you expect application rates to be *increasing* or *decreasing* in `offer_rate`? Why?

*#write answer here*

**Q3.** Use a bin scatter plot to summarize the relationship between `applied` and `offer_rate`, again restricting to customers that are **'High'** risk. [We've written some of the code for you as we have for previous bin scatters.]

In [ ]:
#write code here

#use this table for your plot
offer_rate_table = experiment.where('risk', are.equal_to('High'))

#the code below creates a column that maps each advance value to a decile
rate_bins = pd.qcut(offer_rate_table.column('offer_rate'), 10, labels=False)

**Q4.** What's your eye-ball estimate for the slope? [Later we'll cover a method to measure this slope more scientifically: regression.]

*#To answer, fill in the blank for this sentence: For every 1 percentage point increase in `offer_rate`, loan application rates increase by **???** percentage points.*

We will use this estimate for the slope to benchmark the importance of other advertising content features we investigate below.

### Showing the Interest Rate

Credit Indemnity also randomized whether the letter included an *explicit* mention of the offered interest rate. In cases where the interest rate was suppressed, the information presented in the table was sufficient for the client to calculate the rate. Here is what the two variants look like:

<img src="showint.png" alt="Drawing" style="width: 300px;"/> <img src="noshowint.png" alt="Drawing" style="width: 300px;"/>

In this section we will estimate the average causal effect of showing the interest rate (as measured by `intshown`) on whether a customer applies for a loan (as measured by `applied`).

*But wait, you may ask. We've already used the same data to estimate the average causal effect of the interest rate. Can we estimate average causal effects for two variables in a single experiment?*

**Yes, we can!** That's because for whatever treatment we focus on, the associated treatment and control groups should still be balanced. For example, since both `offer_rate` and `intshown` are randomly assigned, if we compare customers that receive offers with `intshown = 1` and `intshown = 0`, the distribution of offered interest rates should be approximately the same in both groups.

**Run the cell below to compare histograms of `offer_rate` for customers where `intshown = 0` and `intshow = 1`.**

In [ ]:
experiment.select('intshown', 'offer_rate').hist('offer_rate', group = 'intshown')

This looks like only one histogram (rather than two) because the distributions lie exactly on top of one another.

Next we will create the table `intshown_table` to answer the questions below. This table includes only columns we need and will speed up code you run at the end of this lab. **Run the cell below to generate this table.**

In [ ]:
intshown_table = experiment.select('intshown', 'applied')

**Q4.** Calculate the mean of `applied` separately by treatment status. Display using a bar chart or table.

In [ ]:
#write code here

**Q5** Estimate the causal effect of `intshown` on loan application rates by computing the difference in means for `applied` between the treatment and control groups. Store using `intshown_diff`. Does showing the interest rate appear to make a difference?

In [ ]:
#write your code here
intshown_diff = ...
intshown_diff

In [ ]:
_ = ok.grade("q5")

*#write answer here*

### Male vs. Female Photo

Next, we look at the average causal effect of including photo of a woman (versus man) on application behavior.

Here, for simplicity, we will look only at offers that included either a male or female photo. The treatment we will consider is an indicator, `photo_female`, for whether the person in the photo is female. The control group receives offer letters where the person pictured is male. Hence, we're measuring the average causal effect of a female photo, where the counterfactual is a male photo.

<img src="malephoto.png" alt="Drawing" style="width: 100px;"/>
<center>Example of a photo</center>

<img src="femalephoto.png" alt="Drawing" style="width: 100px;"/>
<center>Example of a photo</center>

We will create the table `photo_table` to answer the questions below. **Run the cell below to generate this table.**

In [ ]:
photo_table = experiment.where('photo_none', 0)

**Q6.** Calculate the mean of `applied` separately by treatment status. Display using a bar chart or table.

In [ ]:
#write your code here

**Q7.** Estimate the causal effect of `photo_female` on loan application rates by computing the difference in means for `applied` between the treatment and control groups. Store using `photo_diff`. Does showing the interest rate appear to make a difference? (How does this compare to the effect of an interest rate increase of 1 percentage point?)

In [ ]:
#write your code here
photo_diff = ...
photo_diff

In [ ]:
_ = ok.grade("q7")

*#write answer here*

### Suggested Use for the Loan

Next, we look at the average causal effect of suggesting a specific use for the loan on application behavior. Loan offers were randomly assigned to receive exactly one of the following statements:

1. “You can use this cash for anything you want.”
2. “You can use this cash to pay for school, or for anything else you want.”
3. “You can use this cash to pay off a more expensive debt, or for anything else you want.”
4. “You can use this cash to repair your home, or for anything else you want.”
5. “You can use this cash to buy an appliance, or for anything else you want.”

Here we define the treatment as receiving *any* specific suggested use, meaning any of statements 2 through 5 above. The indicator `suggest_use` is set to one if the offer includes a suggested use, and zero if not(i.e. statement 1 is included).

We will create the table `suggest_table` to answer the questions below. **Run the cell below to generate this table.**

In [ ]:
suggest_table = experiment.select('suggest_use', 'applied')

**Q8.** Calculate the mean of `applied` separately by treatment status. Display using a bar chart or table.

In [ ]:
#write your code here

**Q9.** Estimate the causal effect of `suggest_use` on loan application rates by computing the difference in means for `applied` between the treatment and control groups. Store using `suggest_diff`. Does showing the interest rate appear to make a difference?

In [ ]:
#write your code here
suggest_diff = ...
suggest_diff

In [ ]:
_ = ok.grade("q9")

*#write answer here*

### Advertising a 'Special Offer'

Finally, we look at the average causal effect of advertising the offered interest rate as 'special' or 'low' on application behavior. This corresponds to the indicator `special_offer`.

<img src="specialrate.png" alt="Drawing" style="width: 300px;"/>

We will create the table `special_table` to answer the questions below. **Run the cell below to generate this table.**

In [ ]:
special_table = experiment.where('wave', 2).select('special_offer', 'applied')

**Q10.** Calculate the mean of `applied` separately by treatment status. Display using a bar chart or table.

In [ ]:
#write your code here

**Q11.** Estimate the causal effect of `special_offer` on loan application rates by computing the difference in means for `applied` between the treatment and control groups. Store using `suggest_diff`. Does showing the interest rate appear to make a difference?

In [ ]:
#write your code here
special_diff = ...
special_diff

In [ ]:
_ = ok.grade("q11")

*#write answer here*

### Section 3: Are These Effects Statistically Significant? <a id='tests'></a>

We have measured the difference in application rates between 'treatment' and 'control' groups for a variety of treatments. However, we have not confirmed whether any differences we measured are statistically significant.

In this section, you will test whether each measured treatment effect is statistically significant. In particular, for each treatment, you will conduct a hypothesis test for whether the treatment and control distributions of `applied` are different.

Recall that with hypothesis tests, we need a **null hypothesis** and an **alternative hypothesis**. Recall that our outcome of interest is an indicator whether a customer applies for a loan or not (`applied`). In our case, the null and alternative hypotheses are:

* Null hypothesis: In the population, the distribution of $Y_{0i}$ and $Y_{1i}$ among all customers are the *same*. The difference in the sample is due to chance.

* Alternative hypothesis: In the population, the distribution of $Y_{0i}$ and $Y_{1i}$ among all customers are *different*. [This is a two-sided test.]

The **test statistic** we'll use is the difference in average `applied` between the treatment and control groups.

To predict the test statistic under the null hypothesis, you will use the permutation method.


Here's a [link to the relevant data 8 material.](https://www.inferentialthinking.com/chapters/12/1/ab-testing.html)

Below we have provided the useful function `permuted_sample_average_difference` defined in Data 8.

In [ ]:
#Run this cell to define the function permuted_sample_average_difference 
def permuted_sample_average_difference(table, label, group_label, repetitions):

    tbl = table.select(group_label, label)

    differences = make_array()
    for i in np.arange(repetitions):
        shuffled = tbl.sample(with_replacement = False).column(1)
        original_and_shuffled = tbl.with_column('Shuffled Data', shuffled)

        shuffled_means = original_and_shuffled.group(group_label, np.average).column(2)
        simulated_difference = shuffled_means.item(1) - shuffled_means.item(0)

        differences = np.append(differences, simulated_difference)

    return differences

**Q12.** Use a permutation test to test whether the treatment effect we measured for `special_offer` is statistically significant. (In other words, you will conduct a hypothesis test for whether the treatment and control distributions are different.) We suggest simulation between 100 and 1000 repetitions. Be sure to report a p-value. Is there evidence that including a `special_offer` increases or decreases application rates?

In [ ]:
#assign array of simulated test statistics under null
#hint: use permuted_sample_average_difference function defined above
special_sim_diff = ...

#assign p-value
special_pvalue = ...

print('estimated treatment effect =', special_diff)
print('special value p_value =', special_pvalue)

*#write your answer*

**Q13.** Repeat exercise **Q12**, but for `female_photo`. Is there evidence that including a female photo increases or decreases application rates (relative to a male photo)?

In [ ]:
#assign array of simulated test statistics under null
photo_sim_diff = ...

#assign p-value
photo_pvalue = ...

print('estimated treatment effect =', photo_diff)
print('female photo p_value =', photo_pvalue)

*#write your answer*

**Q14.** Repeat exercise **Q12**, but for `suggest_use`. Is there evidence that suggesting a specific use for the loan increases or decreases application rates?

In [ ]:
#assign array of simulated test statistics under null
suggest_sim_diff = ...

#assign p-value
suggest_pvalue = ...

print('estimated treatment effect =', suggest_diff)
print('suggest use p_value =', suggest_pvalue)

*#write your answer*

**Q15.** Repeat exercise **Q12**, but for `intshown`. Is there evidence that showing the interest rate increases or decreases application rates?

In [ ]:
#assign array of simulated test statistics under null
intshown_sim_diff = ...

#assign p-value
intshown_pvalue = ...

print('estimated treatment effect =', intshown_diff)
print('intshown p_value =', intshown_pvalue)

*#write your answer*

### Section 4: What Matters for Advertising Content? <a id='conclusion'></a>

**Q16.** In conclusion, what matters for advertising in this context? How would you summarize the evidence?

*#write your answer*

----
Congratulations, you've finished Lab 3! To submit the lab, run the two cells below:

In [ ]:
# For your convenience, you can run this cell to run all the tests at once
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

In [ ]:
_ = ok.submit()